## Naïve Bayes – Additional Advice
This notebook guides you through multiple steps you can follow to create a naïve Bayes classifier. After following these steps you will still need to collate and move your code into the main assignment notebook file so that it meets the required format.

Read each step (including the maths!) carefully.

You can implement a naïve Bayes classifier without following this advice.

This notebook will not be graded and does not need to be submitted.

In [1]:
import numpy as np

training_spam = np.loadtxt(open("training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


## The model:  naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* $C = 1$ for spam messages
* $C = 0$ for ham messages


The classifier builds a model for the probability $p(C=c\ |\ \text{message})$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate
$\require{color}$
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \  \textcolor{blue}{p(C=c\ |\ \text{message})}.
\end{equation}
Using Bayes' rule we can write

\begin{equation}
p(C=c\ |\ \text{message}) = \frac{p(\text{message}\ |\ C=c)p(C=c)}{p(\text{message}\ |\ C=1)p(C=1) + p(\text{message}\ |\ C=0)p(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
\textcolor{blue}{p(C=c\ |\ \text{message})} \propto \textcolor{orange}{p(\text{message}\ |\ C=c)}\textcolor{green}{p(C=c)},
\end{equation}

where $\propto$ means "proportional to". The class priors $\textcolor{green}{p(C=c)}$ can be computed directly (you will do so in exercise A) but we need to further simplify $\textcolor{orange}{p(\text{message} \ |\ C=c)}$.


### Choice of the event model: *Multinomial* naïve Bayes

Different naïve Bayes models differ in their distributional assumptions about $\textcolor{orange}{p(\text{message}\ |\ C=c)}$. We represent a message using a **binary** [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as a set of $k$ keywords, that is, $message = (w_1, ..., w_k)$, where $w_i = 1$ if the  keyword $w_i$ appears in the message and $w_i = 0$ otherwise.

We assume that the $p(w_1, ..., w_k |\ C=c)$ follows a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) for each class. Don't let the name scare you, this model simply assigns probabilities to different counts of events with multiple outcomes. So for example: "I roll a biased six-sided die six times, what is the probability that I get each side occurring exactly once" is a question that can be answered with a multinomial distribution. You don't need to understand all of the equations on the Wikipedia page.

Intuitively, the multinomial distribution assumes that the words of a message were "drawn" independently from a bag of $k$ different words. Depending on the class membership $c$, each keyword $w$ has a probability $\theta_{c, w}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy,... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment,... $\}$, if the training data was mostly gathered from emails of researchers.

Under these assumptions, the likelihood of a message, given that it belongs to class $c$, is then proportional to
\begin{equation}
\textcolor{orange}{p(\text{message}\ |\ C=c)} \propto \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}.
\end{equation}


The parameters $\textcolor{brown}{\theta_{c, w}}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$ (add-one smoothing), that is,
\begin{equation}
\textcolor{brown}{\theta_{c, w}} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $n_{c, w}$ is the number of times the keyword $w$ appears in messages of class $c$ in the training set and $n_{c}$ is the total count of keywords for all messages of class $c$, that is, $n_{c} = \sum_w n_{c, w}$.



We are now finally able to rewrite the *maximum a posteriori* estimate in a form that is easy to compute:
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}\right].
\end{equation}


#### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \log \left( \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}}\right)^{w_i} \right) \\
 = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \log( \textcolor{green}{p(C=c)}) + \sum_{i = 1}^k w_i \ \log \left(\textcolor{brown}{\theta_{c, w_i}} \right) \right].
\end{equation}

## Part A: Estimate class priors

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **the logarithm** of the empirical class priors $\textcolor{green}{p(C=c)}$ for $c \in \{0, 1\}$.

In [ ]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
        log(p(C=0)) and log(p(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    ### YOUR CODE HERE...
    return log_class_priors

In [ ]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))

## Part B: Estimate class-conditional likelihoods
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $\log \left(\textcolor{brown}{\theta_{c, w_i}} \right)$ for all words $w_i$ and both classes ($c \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial event model and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [ ]:
def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features (words), calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    ### YOUR CODE HERE...
    return theta


In [ ]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)

## Part  C: Classify e-mails

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

In [ ]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    return class_predictions


In [ ]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))

Now test your `predict` function by classifying messages. You can do this to the *training* data, but you should also try it on the *testing* data. 

In [ ]:
# Check accuracy
true_classes = training_spam[:, 0]
training_set_accuracy = np.mean(np.equal(class_predictions, true_classes))
print(f"Accuracy on the training set: {training_set_accuracy}")

Once you are done, you can move the code into the main assignment notebook.

One way to do this is to follow the rough structure of the class that already exists in that notebook. You can use the `train` method to pass in the data and perform all of the steps before the prediction. You should store data in instance variables, e.g. `self.log_class_priors` and `self.log_class_conditional_likelihoods`. This means that then you can set up the `predict` method to match the one above without needing to pass in the additional variables. **Important:** the predict method must only take a single variable as a parameter (the one called `new_data`) in the skeleton code above.